# Import packages

In [2]:
# Database accessibility 
import pyodbc
from dotenv import dotenv_values 

# Analysis libraries
import pandas as pd 
import numpy as np
from sklearn.impute import SimpleImputer

In [3]:
# Load environment variables from .env file into a dictionary
environment_variables = dotenv_values('.env')


# Get the values for the credentials you set in the '.env' file
database = environment_variables.get("database")
server = environment_variables.get("server")
username = environment_variables.get("user")
password = environment_variables.get("password")


connection_string = f"DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}"



In [4]:
# Use the connect method of the pyodbc library and pass in the connection string.
# This will connect to the server and might take a few seconds to be complete. 
# Check your internet connection if it takes more time than necessary

connection = pyodbc.connect(connection_string)

In [5]:
# Define SQL queries for each table
query1 = 'SELECT * FROM dbo.oil'
query2 = 'SELECT * FROM dbo.holidays_events'
query3 = 'SELECT * FROM dbo.stores'

# Read data from tables into pandas DataFrames
oil = pd.read_sql(query1, connection)
holidays_events = pd.read_sql(query2, connection)
stores = pd.read_sql(query3, connection)

# Close the database connection
connection.close()

C:\Users\hp\AppData\Local\Temp\ipykernel_3068\3886160679.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  oil = pd.read_sql(query1, connection)
C:\Users\hp\AppData\Local\Temp\ipykernel_3068\3886160679.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  holidays_events = pd.read_sql(query2, connection)
C:\Users\hp\AppData\Local\Temp\ipykernel_3068\3886160679.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  stores = pd.read_sql(query3, connection)


In [6]:
oil.head()

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.139999
2,2013-01-03,92.970001
3,2013-01-04,93.120003
4,2013-01-07,93.199997


Rename second col to dailly oil prices

In [7]:
holidays_events.head()
holidays_events.rename(columns={
    'type': 'holiday_type'
}, inplace=True)

# Print the resulting DataFrame
holidays_events

,date,holiday_type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False
...,...,...,...,...,...,...
345,2017-12-22,Additional,National,Ecuador,Navidad-3,False
346,2017-12-23,Additional,National,Ecuador,Navidad-2,False
347,2017-12-24,Additional,National,Ecuador,Navidad-1,False
348,2017-12-25,Holiday,National,Ecuador,Navidad,False


Rename type here as holiday type and concat with oil['dailyoilprices']

In [8]:
stores.head()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [9]:
sample_submission = pd.read_csv('data/sample_submission.csv')
sample_submission.head()

,id,sales
0,3000888,0.0
1,3000889,0.0
2,3000890,0.0
3,3000891,0.0
4,3000892,0.0


Do away with this set since sales has no values.

In [10]:
transactions = pd.read_csv('data/transactions.csv')
transactions.head()

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922


In [11]:
train = pd.read_csv('data/train.csv')
train.sample(5)

,id,date,store_nbr,family,sales,onpromotion
2861950,2861950,2017-05-30,10,PERSONAL CARE,195.0,8
54409,54409,2013-01-31,35,PERSONAL CARE,102.0,0
1863826,1863826,2015-11-14,54,LADIESWEAR,0.0,0
2857140,2857140,2017-05-27,26,AUTOMOTIVE,9.0,0
1497911,1497911,2015-04-23,38,DAIRY,568.0,0


In [12]:
train[(train['sales'] == 770) & (train['store_nbr'] == 25) ]

,id,date,store_nbr,family,sales,onpromotion
250053,250053,2013-05-21,25,GROCERY I,770.0,0
417552,417552,2013-08-23,25,BEVERAGES,770.0,0
631396,631396,2013-12-21,25,CLEANING,770.0,0


...

In [13]:
# oil.to_csv('data/oil.csv',index=False)
# transactions.to_csv('data/transactions.csv',index=False)
# holidays_events.to_csv('data/holidays_events.csv',index=False)
# stores.to_csv('data/stores.csv',index=False)

## Join Tables

### Join to display data contained in both dataframes

In [14]:
full_transaction= pd.merge(transactions, train)
full_transaction.sample(5)

,date,store_nbr,transactions,id,family,sales,onpromotion
87938,2013-02-28,48,2268,104768,PET SUPPLIES,0.0,0
2627441,2017-06-05,20,1605,2872994,HARDWARE,1.0,0
492263,2013-11-18,35,559,572948,BEAUTY,0.0,0
466301,2013-11-01,48,2714,543125,FROZEN FOODS,140.0,0
1378073,2015-06-08,45,3315,1580165,PET SUPPLIES,17.0,0


## Join the full transactions based on stores

In [15]:
result = pd.merge(full_transaction, stores, on='store_nbr', how='inner')
result.head(5)


,date,store_nbr,transactions,id,family,sales,onpromotion,city,state,type,cluster
0,2013-01-01,25,770,561,AUTOMOTIVE,0.0,0,Salinas,Santa Elena,D,1
1,2013-01-01,25,770,562,BABY CARE,0.0,0,Salinas,Santa Elena,D,1
2,2013-01-01,25,770,563,BEAUTY,2.0,0,Salinas,Santa Elena,D,1
3,2013-01-01,25,770,564,BEVERAGES,810.0,0,Salinas,Santa Elena,D,1
4,2013-01-01,25,770,565,BOOKS,0.0,0,Salinas,Santa Elena,D,1


## Join the full transactions based on oil data for each date

In [16]:
result1= pd.merge(result, oil, on='date', how='inner')
result1.sample(5)


,date,store_nbr,transactions,id,family,sales,onpromotion,city,state,type,cluster,dcoilwtico
685589,2014-09-19,44,4057,1115018,HARDWARE,1.0,0,Quito,Pichincha,A,5,92.430000
1292824,2016-02-22,35,705,2039548,HOME AND KITCHEN II,12.0,0,Playas,Guayas,C,3,31.370001
438802,2014-02-07,12,1062,714682,BABY CARE,0.0,0,Latacunga,Cotopaxi,C,15,99.980003
709817,2014-10-13,3,2846,1157264,LAWN AND GARDEN,22.0,0,Quito,Pichincha,D,8,85.730003
1377969,2016-04-29,15,1304,2158221,LINGERIE,4.0,0,Ibarra,Imbabura,C,15,45.980000


## Join the full transactions based on holidays

In [17]:
salesdata= pd.merge(result1, holidays_events, on='date', how='inner')
salesdata.reset_index(drop=True,inplace=True)
salesdata.head(5)


,date,store_nbr,transactions,id,family,sales,onpromotion,city,state,type,cluster,dcoilwtico,holiday_type,locale,locale_name,description,transferred
0,2013-01-01,25,770,561,AUTOMOTIVE,0.0,0,Salinas,Santa Elena,D,1,NaN,Holiday,National,Ecuador,Primer dia del ano,False
1,2013-01-01,25,770,562,BABY CARE,0.0,0,Salinas,Santa Elena,D,1,NaN,Holiday,National,Ecuador,Primer dia del ano,False
2,2013-01-01,25,770,563,BEAUTY,2.0,0,Salinas,Santa Elena,D,1,NaN,Holiday,National,Ecuador,Primer dia del ano,False
3,2013-01-01,25,770,564,BEVERAGES,810.0,0,Salinas,Santa Elena,D,1,NaN,Holiday,National,Ecuador,Primer dia del ano,False
4,2013-01-01,25,770,565,BOOKS,0.0,0,Salinas,Santa Elena,D,1,NaN,Holiday,National,Ecuador,Primer dia del ano,False


## Drop some columns (id column)

In [18]:
# salesdata.drop(columns='id', inplace=True)

## Rename columns

In [19]:
salesdata.rename(columns={
    'store_nbr': 'store_number',
    'dcoilwtico': 'oil_prices',
}, inplace=True)
salesdata.sample(5)

,date,store_number,transactions,id,family,sales,onpromotion,city,state,type,cluster,oil_prices,holiday_type,locale,locale_name,description,transferred
51592,2014-04-14,18,1530,832504,GROCERY II,24.0,0,Quito,Pichincha,B,16,104.050003,Holiday,Local,Libertad,Cantonizacion de Libertad,False
149541,2015-08-05,10,1014,1682259,HOME CARE,178.0,0,Quito,Pichincha,C,15,45.130001,Holiday,Local,Esmeraldas,Fundacion de Esmeraldas,False
81919,2014-07-24,2,1798,1012552,GROCERY II,43.0,0,Quito,Pichincha,D,13,102.760002,Additional,Local,Guayaquil,Fundacion de Guayaquil-1,False
131436,2015-04-21,9,2134,1495095,PRODUCE,0.0,0,Quito,Pichincha,B,6,55.580002,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False
164212,2015-11-06,45,3391,1849225,BOOKS,0.0,0,Quito,Pichincha,A,11,44.320000,Holiday,Regional,Santo Domingo de los Tsachilas,Provincializacion de Santo Domingo,False


In [20]:
salesdata.columns

Index(['date', 'store_number', 'transactions', 'id', 'family', 'sales',
       'onpromotion', 'city', 'state', 'type', 'cluster', 'oil_prices',
       'holiday_type', 'locale', 'locale_name', 'description', 'transferred'],
      dtype='object')

In [21]:
salesdata = salesdata[['id', 'date',  'store_number', 'transactions', 'family', 'sales',
       'onpromotion', 'city', 'state', 'type', 'cluster', 'oil_prices',
       'holiday_type', 'locale', 'locale_name', 'description', 'transferred']]

In [22]:
salesdata.head()

,id,date,store_number,transactions,family,sales,onpromotion,city,state,type,cluster,oil_prices,holiday_type,locale,locale_name,description,transferred
0,561,2013-01-01,25,770,AUTOMOTIVE,0.0,0,Salinas,Santa Elena,D,1,NaN,Holiday,National,Ecuador,Primer dia del ano,False
1,562,2013-01-01,25,770,BABY CARE,0.0,0,Salinas,Santa Elena,D,1,NaN,Holiday,National,Ecuador,Primer dia del ano,False
2,563,2013-01-01,25,770,BEAUTY,2.0,0,Salinas,Santa Elena,D,1,NaN,Holiday,National,Ecuador,Primer dia del ano,False
3,564,2013-01-01,25,770,BEVERAGES,810.0,0,Salinas,Santa Elena,D,1,NaN,Holiday,National,Ecuador,Primer dia del ano,False
4,565,2013-01-01,25,770,BOOKS,0.0,0,Salinas,Santa Elena,D,1,NaN,Holiday,National,Ecuador,Primer dia del ano,False
